In [7]:
# Import packages
import sys
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from natsort import natsorted
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer,Input, Dense, Dropout, Activation, Concatenate, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.compat.v1 import InteractiveSession
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import load_model
import tensorflow.keras.metrics
####import tensorflow_addons as tfa not required
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical, plot_model
# from pycm import *
#import matplotlib.pyplot as plt

#from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve,roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.metrics import r2_score

### 

# Import packages
import sys
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from natsort import natsorted
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer,Input, Dense, Dropout, Activation, Concatenate, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.compat.v1 import InteractiveSession
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tensorflow.keras.metrics
####import tensorflow_addons as tfa not required
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical, plot_model
# from pycm import *
#import matplotlib.pyplot as plt

#from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve,roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.metrics import r2_score


In [8]:
P = pd.read_csv('data/ProtSeqForModel.csv')
D = pd.read_csv('data/DrugSeqForModel.csv')

drug_smiles = pd.read_csv('data/drug_smiles.csv')
DTI_index = pd.read_csv('data/DTI_index.csv')

In [11]:
def integer_encoding(data, dict_):
    """
    - Encodes code sequence to integer values.
    - 20 common amino acids are taken into consideration
    and rest 4 are categorized as 0.
    """
    encode_list = []
    for row in data['seq'].values:
        row_encode = []
        for code in row:
            row_encode.append(dict_.get(code, 0))
        encode_list.append(np.array(row_encode))
    return encode_list

def create_dict(codes):
    char_dict = {}
    for index, val in enumerate(codes):
        char_dict[val] = index+1
    return char_dict

# Encode amino acides and smile characters
codes_target = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
char_dict_target = create_dict(codes_target)

codes_drug = [char for char in ''.join(set(''.join(drug_smiles['canonical_smiles'].values)))]
char_dict_drug = create_dict(codes_drug)

encode_target = integer_encoding(P, char_dict_target) 
encode_drug = integer_encoding(D, char_dict_drug) 

# padding sequences
max_length_target = 1000
max_length_drug = 150
pad_target = pad_sequences(encode_target, maxlen=max_length_target, padding='post', truncating='post')
pad_drug = pad_sequences(encode_drug, maxlen=max_length_drug, padding='post', truncating='post')

# label/integer encoding output variable: (y)
le = LabelEncoder()
y = le.fit_transform(DTI_index['activity'].tolist())
# One hot encoding of outputs
y = to_categorical(y)

In [16]:
pad_target[0]

array([11, 16, 14,  4, 15, 13, 17,  5, 20, 15, 14,  4, 10, 12,  9, 17,  8,
       19,  4, 18, 13,  4, 15, 20, 14, 12, 10, 16, 13, 18,  6, 16,  6,  1,
       20,  6, 16, 18,  2,  1,  1,  5,  3, 17,  9, 17,  6, 10, 15, 18,  1,
       18,  9,  9, 10, 16, 15, 13,  5, 14, 16,  8,  8,  7,  1,  9, 15, 17,
       20, 15,  4, 10, 15, 10, 10,  9,  7, 11,  9,  7,  4, 12, 18,  8,  6,
       10, 10,  3, 18,  5, 17, 13,  1, 15, 16, 10,  4,  4,  5, 12,  3, 18,
       20, 10, 18, 17,  7, 10, 11,  6,  1,  3, 10, 12, 12,  8, 18,  9,  2,
       14,  9, 10, 17,  3,  3,  7, 18, 14,  5, 10,  8, 20, 14,  8, 10, 15,
        6, 10,  9, 20,  8,  7, 16,  1,  3,  8,  8,  7, 15,  3, 10,  9, 13,
       16, 12, 10,  1, 18, 12,  4,  3,  2,  4, 10,  9,  8, 10,  3,  5,  6,
       10,  1, 15,  7, 17,  3,  3,  4, 11, 17,  6, 20, 18,  1, 17, 15, 19,
       20, 15,  1, 13,  4,  8, 11, 10, 12, 19, 11,  7, 20, 12, 14, 17, 18,
        3,  8, 19, 16, 18,  6,  2,  8, 11,  1,  4, 10, 10, 17,  6, 15, 17,
       10,  5, 13,  6, 17